# Curve Weekly Opportunities & Metrics HTML Generator Notebook

## Imports and Constants

In [3]:
import pandas as pd
import numpy as np
from ast import literal_eval
import requests
import time
from datetime import datetime, timedelta
import pytz
from IPython.display import HTML
import random

In [4]:
POOL_TYPES = {
    'factory-twocrypto': 'cryptoswap',
    'factory-crvusd': 'stableswap',
    'factory-eywa': 'stableswap',
    'factory-stable-ng': 'stableswap', 
    'main': 'stableswap', 
    'factory': 'stableswap', 
    'factory-crypto': 'cryptoswap', 
    'crypto': 'cryptoswap', 
    'factory-tricrypto': 'cryptoswap',
}

POOL_TYPES_ASSETS = {
    'usd': ['usdc', 'usdt', 'usdm', 'crvusd', 'dola', 'dai', 'sdai', 'sdola', 'money', 'usds', 'susds', 'frax', 'lusd', 'susde', 'usde', 'gho', 'mim', 'susd', 'dyad', 's3CRV_e', 'sUSDC_m', 'scrvusd'],
    'btc': ['wbtc', 'renbtc', 'sbtc', 'hbtc', 'tbtc', 'obtc', 'pbtc', 'ibtc', 'bbtc', 'dbtc', 'ebtc', 'fbtc', 'gbtc', 'hbtc', 'jbtc', 'kbtc', 'lbtc', 'mbtc', 'nbtc', 'obtc', 'pbtc', 'qbtc', 'rbtc', 'sbtc', 'tbtc', 'ubtc', 'vbtc', 'wbtc', 'xbtc', 'ybtc', 'zbtc', 'abtc', 'bbtc', 'cbtc', 'dbtc', 'ebtc', 'fbtc', 'gbtc', 'hbtc', 'ibtc', 'jbtc', 'kbtc', 'lbtc', 'mbtc', 'nbtc', 'obtc', 'pbtc', 'qbtc', 'rbtc', 'sbtc', 'tbtc', 'ubtc', 'vbtc', 'wbtc', 'xbtc', 'ybtc', 'zbtc', 'abtc', 'bbtc', 'cbtc', 'dbtc', 'ebtc', 'fbtc', 'gbtc', 'hbtc', 'ibtc', 'jbtc', 'kbtc', 'lbtc', 'mbtc', 'nbtc', 'obtc', 'pbtc', 'qbtc', 'rbtc', 'sbtc', 'tbtc', 'ubtc', 'vbtc', 'wbtc', 'xbtc', 'ybtc', 'zbtc', 'abtc', 'bbtc', 'cbtc', 'dbtc', 'ebtc', 'fbtc', 'gbtc', 'hbtc', 'ibtc', 'jbtc', 'kbtc', 'lbtc', 'mbtc', 'nbtc', 'obtc', 'pbtc', 'qbtc', 'rbtc', 'sbtc', 'tbtc', 'ubtc', 'vbtc', 'wbtc', 'xbtc', 'ybtc', 'zbtc', 'abtc', 'bbtc', 'cbtc', 'dbtc', 'ebtc', 'fbtc', 'gbtc', 'hbtc', 'ibtc', 'jbtc', 'kbtc', 'lbtc', 'mbtc', 'nbtc', 'obtc', 'pbtc', 'qbtc', 's2BTC_e'],
    'crv': ['crv', 'sdcrv', 'asdcrv'],
    'eth': ['eth', 'weth', 'steth', 'frxeth', 'wsteth', 'seth', 'aleth', 'reth', 'oeth', 'sWETH_e'],
    'eur': ['eura', 'eurt', 'eurs', 'eurc', 'eure', 'ceur', 'seur']
}

POOL_TYPES_ASSET_DICT = {
    asset.lower(): pool_type 
    for pool_type, assets in POOL_TYPES_ASSETS.items() 
    for asset in assets
}

# Extra broken pools I hide
MY_HIDDEN_POOLS_FILTER = {
    '0x833e4b9740B4C73cC4870D28492A296d005E2895': 'polygon',
    '0xF1005aC82E89dE676449525c3Cb86959216864C1': 'ethereum',
    '0x29a3d66b30bc4ad674a4fdaf27578b64f6afbfe7': 'optimism',
    '0xdadd23929ca8efcbc43aaf8f677d426563cc40d7': 'arbitrum',
    '0x004C167d27ADa24305b76D80762997Fa6EB8d9B2': 'ethereum',
}

CRVUSD_MARKET_LINKS = {
    'all': {'link': 'https://crvusd.curve.fi/#/ethereum/markets', 'name': 'All'},
    '0x8472A9A7632b173c8Cf3a86D3afec50c35548e76': {'link': 'https://crvusd.curve.fi/#/ethereum/markets/sfrxeth/create', 'name': 'sfrxETH (old)'},
    '0xEC0820EfafC41D8943EE8dE495fC9Ba8495B15cf': {'link': 'https://crvusd.curve.fi/#/ethereum/markets/sfrxeth2/create', 'name': 'sfrxETH'},
    '0x100dAa78fC509Db39Ef7D04DE0c1ABD299f4C6CE': {'link': 'https://crvusd.curve.fi/#/ethereum/markets/wsteth/create', 'name': 'wstETH'},
    '0x4e59541306910aD6dC1daC0AC9dFB29bD9F15c67': {'link': 'https://crvusd.curve.fi/#/ethereum/markets/wbtc/create', 'name': 'WBTC'},
    '0xA920De414eA4Ab66b97dA1bFE9e6EcA7d4219635': {'link': 'https://crvusd.curve.fi/#/ethereum/markets/weth/create', 'name': 'WETH'},
    '0x1C91da0223c763d2e0173243eAdaA0A2ea47E704': {'link': 'https://crvusd.curve.fi/#/ethereum/markets/tbtc/create', 'name': 'tBTC'},
}

CRVUSD_ADDR_LIST = {
    'scrvUSD': '0x0655977FEb2f289A4aB78af67BAB0d17aAb84367',
    'crvUSD': '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E',
}

In [5]:
HTML_STYLING = f"""<style>

.negative-change {{
  color: red;
}}

.metric-table td:first-child {{
    background-color: #F7F7F7;
}}

.chain-column td:first-child {{
    width: 50px;
}}

.crvusd-metric-column td:first-child {{
    min-width: 250px;
}}

.pool-column td:nth-child(2) {{
    min-width: 150px;
}}
  
.gh-content table.curve-table {{
    width: 100% !important;
    max-width: 100% !important;
    margin: 0 auto !important;
    display: table !important;
}}
  
.curve-table-container {{
    position: relative !important;
    width: 100% !important;
    overflow-x: auto !important;
    -webkit-overflow-scrolling: touch !important;
    margin-bottom: 1rem !important;
}}

.curve-table {{
    border-collapse: collapse !important;
    width: 100% !important;
    max-width: 100% !important;
    margin-left: auto !important;
    margin-right: auto !important;
    vertical-align: middle;
}}

.gh-content.gh-canvas {{
  scrollbar-width: none; /* Firefox */
  -ms-overflow-style: none; /* IE and Edge */
}}

.gh-content.gh-canvas::-webkit-scrollbar {{
  display: none; /* WebKit (Chrome/Safari) */
}}

.curve-table .network-cell {{
  display: flex;
  align-items: center;
  gap: 8px;
  width: fit-content;
  margin: 0 auto;
  vertical-align: middle;
}}

.curve-table .metrics-name-cell {{
  display: flex;
  align-items: left;
  gap: 8px;
  width: fit-content;
  vertical-align: middle;
}}

.curve-table .network-icon {{
  width: 24px;
  height: 24px;
  flex-shrink: 0;
}}

.curve-table .pool-link {{
  text-decoration: none !important;
  color: inherit;
  display: block;
  width: 100%;
  vertical-align: middle;
}}

.curve-table .pool-content {{
  display: flex;
  flex-wrap: wrap;
  gap: 8px;
  width: fit-content;
  max-width: 100%;
  vertical-align: middle;
}}

.curve-table .token-pill-pool {{
  background-color: #EEECEB;
}}

.curve-table .token-pill-borrow {{
  background-color: #BBEDC0;
  
}}

.curve-table .token-pill-collateral {{
  background-color: #DEDBD8;
}}

.token-pill-pool, .token-pill.collateral, .token-pill-borrow{{
  font-weight: bold !important;
  display: inline-flex;
  flex: 0 0 auto;
  align-items: center;
  gap: 4px;
  width: fit-content;
  white-space: nowrap;
  text-decoration: none !important;
  padding: 4px 8px;
  border-radius: 99999px;
}}

.token-pill-pool:hover, .token-pill.collateral:hover, .token-pill-borrow:hover{{
  background-color: #DEDBD8 !important;
  font-weight: bold !important;
  
}}

.curve-table .token-icon {{
  width: 24px;
  height: 24px;
  border-radius: 50%;
  flex-shrink: 0;
}}

/* Scope the th and td styles to our specific table */
.curve-table th,
.curve-table td {{
  text-align: left;
  padding: 8px;
  border: 1px solid #e0e0e0;
  vertical-align: middle;
}}

a{{
  text-decoration: none;
}}

a:hover{{
}}

thead{{
  background-color: #DEDBD8 !important;
  text-align: left;
  font-weight: bold !important;
}}

th{{
  background-color: #DEDBD8 !important; 
}}

tr th:last-child{{
  text-align: right !important;
}}

tr td:first-child{{
  font-weight: bold;
}}
tr td:last-child{{
  font-weight: bold;
  text-align: right !important;
}}

.curve-table th {{
  font-weight: bold;
  background-color: #f6f6f6;
}}


.curve-table td a {{
    text-decoration: none !important;
    color: inherit !important;
}}

.curve-table td a:hover {{
     color: #1F1E1B !important;
}}
  
.curve-table td .pool-content {{
  width: fit-content !important;
  display: flex !important;
  vertical-align: middle;
}}

.curve-table td .token-pill-pool {{
  width: fit-content !important;
  flex: 0 0 auto !important;
  vertical-align: middle;
}}

.curve-table td .token-pill-borrow {{
  width: fit-content !important;
  flex: 0 0 auto !important;
  vertical-align: middle;
}}

.curve-table td .token-pill-collateral {{
  width: fit-content !important;
  flex: 0 0 auto !important;
  vertical-align: middle;
}}
</style>"""

# Function Declarations

### Misc

In [22]:
def epoch_time_formatter(str_time):
    try:
        dt = datetime.strptime(str_time, '%Y-%m-%dT%H:%M:%S')
    except:
        dt = datetime.strptime(str_time, '%Y-%m-%dT%H:%M:%S.%f')
    return int(dt.timestamp())


def get_cur_time():
    return int(time.time())


def url_exists(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False


def format_number(x, sig_figures=4, max_decimals=2, use_suffixes=True):
    if not use_suffixes:
        # Original formatting logic
        sig_fig_str = f'{x:,.{sig_figures}g}'
        num = float(sig_fig_str.replace(',', ''))
        formatted = f'{num:,.{max_decimals}f}'
        if '.' in formatted:
            formatted = formatted.rstrip('0').rstrip('.')
        return formatted
    
    # Define suffix thresholds
    suffixes = [
        (1e9, 'B'),  # billion
        (1e6, 'M'),  # million
        (1e3, 'k'),  # thousand
        (1, '')      # default - no suffix
    ]
    
    # Find appropriate suffix
    for threshold, suffix in suffixes:
        if abs(x) >= threshold:
            # Scale the number down
            scaled_num = x / threshold
            # Format with sig figures
            sig_fig_str = f'{scaled_num:.{sig_figures}g}'
            num = float(sig_fig_str)
            # Format with max decimals
            formatted = f'{num:.{max_decimals}f}'
            # Strip unnecessary zeros and decimal point
            if '.' in formatted:
                formatted = formatted.rstrip('0').rstrip('.')
            return f'{formatted}{suffix}'
    
    # For numbers less than 1
    sig_fig_str = f'{x:.{sig_figures}g}'
    num = float(sig_fig_str)
    formatted = f'{num:.{max_decimals}f}'
    if '.' in formatted:
        formatted = formatted.rstrip('0').rstrip('.')
    return formatted

### Curve Prices API

In [23]:
def prices_api_request(method):
    base_url = "https://prices.curve.fi"
    url = base_url + method
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to retrieve data. Status code: {response.status_code}")
    

def get_all_chains():
    data = prices_api_request("/v1/chains/")
    chains = []
    for item in data['data']:
        chains.append(item["name"])
    return chains


def get_chain_tx_count(start, end):
    url = f"/v1/chains/activity/transactions?start={start-(end-start)}&end={end}"
    data = prices_api_request(url)
    tx_counts = {'all': {}}

    # go through each chain in the data set
    for raw_chain in data['data']:
        tx_counts[raw_chain['chain']] = {}
        chain_data = tx_counts[raw_chain['chain']]
        for raw_day in raw_chain['transactions']:
            app_type = raw_day['type']
            ts = epoch_time_formatter(raw_day['timestamp'])
            if app_type not in chain_data:
                chain_data[app_type] = {'cur_week': [], 'prev_week': [], 'cur_week_total': 0, 'prev_week_total': 0}
            if end-ts > 86400 and ts >= start-86400:
                chain_data[app_type]['cur_week'].append(raw_day['transactions'])
                chain_data[app_type]['cur_week_total'] += raw_day['transactions']
            elif start-ts > 86400 and ts >= (start-(end-start))-86400:
                chain_data[app_type]['prev_week'].append(raw_day['transactions'])
                chain_data[app_type]['prev_week_total'] += raw_day['transactions']
        for app_type in chain_data:
            if app_type not in tx_counts['all']:
                tx_counts['all'][app_type] = {
                    'cur_week_total': chain_data[app_type]['cur_week_total'], 
                    'prev_week_total': chain_data[app_type]['prev_week_total']
                }
            else:
                tx_counts['all'][app_type]['cur_week_total'] += chain_data[app_type]['cur_week_total']
                tx_counts['all'][app_type]['prev_week_total'] += chain_data[app_type]['prev_week_total']
    return tx_counts


def get_chain_user_stats(start, end):
    url = f"/v1/chains/activity/users?start={start-(end-start)}&end={end}"
    data = prices_api_request(url)
    user_stats = {}

    # go through each chain in the data set
    for raw_chain in data['data']:
        user_stats[raw_chain['chain']] = {}
        chain_data = user_stats[raw_chain['chain']]
        for raw_day in raw_chain['users']:
            user_type = raw_day['type']
            ts = epoch_time_formatter(raw_day['timestamp'])
            if user_type not in chain_data:
                chain_data[user_type] = {'cur_week': [], 'prev_week': []}
            if end-ts > 86400 and ts >= start-86400:
                chain_data[user_type]['cur_week'].append(raw_day['users'])
            elif start-ts > 86400 and ts >= (start-(end-start))-86400:
                chain_data[user_type]['prev_week'].append(raw_day['users'])

    return user_stats


def get_chain_pools(chain):
    chain_pools = {}
    url = f"/v1/chains/{chain}"
    data = prices_api_request(url)

    chain_pools['tvl'] = data['total']['total_tvl']

    chain_pools['contracts'] = {}
    for contract in data['data']:
        address = contract['address']

        chain_pools['contracts'][address] = { \
            'name': contract['name'], \
            'address': address, \
            'tvl_usd': contract['tvl_usd'], \
            'base_weekly_apr': contract['base_weekly_apr'], \
            'coins': contract['coins'], \
            'type': 'pool' \
        }
    
    return chain_pools


def get_all_chain_pools(chains):
    all_pools = {}
    for chain in chains:
        all_pools[chain] = get_chain_pools(chain)
    
    return all_pools


def get_chain_lending_markets(chain):
    chain_lending_markets = {}
    url = f"/v1/lending/markets/{chain}"
    data = prices_api_request(url)
    
    if data['count'] == 0:
        return None

    chain_lending_markets['contracts'] = {}
    for contract in data['data']:
        controller = contract['controller']

        if contract['total_debt_usd'] == 0:
            utilization = 0
        else:
            utilization = float(contract['total_debt_usd']) / contract['total_assets_usd']

        available_assets = contract['total_assets'] * (1 - utilization)
        available_assets_usd = contract['total_assets_usd'] - contract['total_debt_usd']

        chain_lending_markets['contracts'][controller] = { \
            'name': contract['name'], \
            'controller': controller, \
            'vault': contract['vault'], \
            'llamma': contract['llamma'], \
            'policy': contract['policy'], \
            'oracle': contract['oracle'], \
            'oracle_pools': contract['oracle_pools'], \
            'rate': contract['rate'], \
            'borrow_apy': contract['borrow_apy'], \
            'lend_apy': contract['lend_apy'], \
            'n_loans': contract['n_loans'], \
            'total_assets': contract['total_assets'], \
            'total_assets_usd': contract['total_assets_usd'], \
            'collateral_balance': contract['collateral_balance'], \
            'collateral_balance_usd': contract['collateral_balance_usd'], \
            'total_debt': contract['total_debt'], \
            'total_debt_usd': contract['total_debt_usd'], \
            'available_assets': available_assets, \
            'available_assets_usd': available_assets_usd, \
            'utilization': utilization, \
            'collateral_token': contract['collateral_token'], \
            'borrowed_token': contract['borrowed_token'], \
            'type'  : 'lending' \
        }
        pass
    
    return chain_lending_markets


def get_all_chain_lending_markets(chains):
    all_chains_lending_markets = {}
    for chain in chains:
        all_chains_lending_markets[chain] = get_chain_lending_markets(chain)
    
    return all_chains_lending_markets


def get_crvusd_markets():
    url = "/v1/crvusd/markets/ethereum"
    data = prices_api_request(url)
    crvusd_markets = {}

    crvusd_markets = {}
    for contract in data['data']:
        address = contract['address']
        crvusd_markets[address] = { \
            'address': address, \
            'llamma': contract['llamma'], \
            'collateral_token': contract['collateral_token'], \
            'stablecoin_token': contract['stablecoin_token'], \
            'type'  : 'crvusd' \
        }
    
    return crvusd_markets


def get_pool_volume(chain, address, start, end):
    prev_start = start - (end - start)
    prev_end = start
    url = f"/v1/volume/usd/{chain}/{address}?interval=day&start={prev_start}&end={end}"
    data = prices_api_request(url)
    volume_usd = 0
    fees_usd = 0
    prev_volume_usd = 0
    prev_fees_usd = 0
    if data['data'] != []:
        for day in data['data']:
            ts = day['timestamp']
            if ts > start-86400 and ts <= end-86400:
                #print(f"Volume Data TS: {datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')} think it's within start and end: {datetime.fromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S')} and {datetime.fromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S')}")
                volume_usd += day['volume']
                fees_usd += day['fees']
            elif ts > (prev_start-86400) and ts <= prev_end-86400:
                #print(f"Volume Data TS: {datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')} think it's within prev start and end: {datetime.fromtimestamp(prev_start).strftime('%Y-%m-%d %H:%M:%S')} and {datetime.fromtimestamp(prev_end).strftime('%Y-%m-%d %H:%M:%S')}")
                prev_volume_usd += day['volume']
                prev_fees_usd += day['fees']
    return volume_usd, fees_usd, prev_volume_usd, prev_fees_usd


def get_tvl_for_period(chain, address, start, end):
    tvl_data = {    
            'tvl': 0,
            'tvl_snapshot': None,
            'token_supply': 0,
            'token_supply_snapshot': None,
            'virtual_price_snapshot': None,
        }

    url = f"/v1/snapshots/{chain}/{address}/tvl?start={start}&end={end}&unit=none"
    data = prices_api_request(url)
    if data['data'] != []:
        for item in data['data']:
            if item['tvl_usd'] != None and tvl_data['tvl_snapshot'] == None:
                tvl_data['tvl'] = item['tvl_usd']
                tvl_data['tvl_snapshot'] = item
            if item['token_supply'] != None and tvl_data['token_supply_snapshot'] == None:
                tvl_data['token_supply'] = item['token_supply']
                tvl_data['token_supply_snapshot'] = item
            if tvl_data['tvl_snapshot'] != None and tvl_data['token_supply_snapshot'] != None:
                break
    
        if tvl_data['tvl_snapshot'] == None and tvl_data['token_supply_snapshot'] != None:
            # couldn't get usd tvl from the snapshot, try getting it from virtual price
            url = f"/v1/snapshots/{chain}/{address}?start={start}&end={end}&unit=none"
            pool_virtual_data = prices_api_request(url)
            for item in pool_virtual_data['data']:
                if item['virtual_price'] != None:
                    tvl_data['tvl'] = tvl_data['token_supply'] * item['virtual_price'] / 10**18
                    tvl_data['virtual_price_snapshot'] = item
                    break
    return tvl_data
    

def get_tvl(chain, address, creationTs, start, end=None):
    tvl_data = {
        'start': None,
        'end': None,
    }

    # get tvl data for the pool
    tvl_data['start'] = get_tvl_for_period(chain, address, creationTs, start)
    if end != None:
        tvl_data['end'] = get_tvl_for_period(chain, address, creationTs, end)
    
    # check if we have tvl data for the pool
    if  tvl_data['start']['tvl_snapshot'] == None and tvl_data['start']['virtual_price_snapshot'] == None:
        if  tvl_data['end']['tvl_snapshot'] == None and tvl_data['end']['virtual_price_snapshot'] == None:
            raise Exception(f"Error: no tvl data found for pool {address} on chain {chain}")
        else:
            print(f"Warning: no tvl data found for pool {address} on chain {chain} for start period {start}, new pool?")

    return tvl_data


def prices_get_lending_snapshots(chain, address, start, end):
    url = f"/v1/lending/markets/{chain}/{address}/snapshots?fetch_on_chain=true&agg=day"
    data = prices_api_request(url)
    prev_snapshot = None
    cur_snapshot = None
    for snapshot in data['data']:
        ts = epoch_time_formatter(snapshot['timestamp'])
        if ts <= start and start - ts < 86400:
            prev_snapshot = snapshot
        elif ts <= end and end - ts < 86400:
            cur_snapshot = snapshot
    return prev_snapshot, cur_snapshot


def aggregate_crvusd_snapshots(base_items, start_index, end_index):
    """
    Aggregate metrics with custom functions for each field over a specified range.
    
    Args:
        items: List of dictionaries containing the metrics
        start_index: Starting index (inclusive)
        end_index: Ending index (inclusive)
    
    Returns:
        Dictionary with aggregated values for each field using appropriate functions
    """
    items = base_items.copy()
    if not items or start_index > end_index or start_index < 0 or end_index >= len(items):
        return None
    
    selected_items = items[start_index:end_index + 1]
    count = len(selected_items)
    
    # Define aggregation functions for each field
    aggregations = {
        'rate': lambda x: sum(item['rate'] for item in x) / count,
        'minted': lambda x: max(item['minted'] for item in x) / count,
        'redeemed': lambda x: max(item['redeemed'] for item in x) / count,
        'total_collateral': lambda x: sum(item['total_collateral'] for item in x) / count,
        'total_collateral_usd': lambda x: sum(item['total_collateral_usd'] for item in x) / count,
        'total_stablecoin': lambda x: sum(item['total_stablecoin'] for item in x) / count,
        'total_debt': lambda x: sum(item['total_debt'] for item in x) / count,
        'n_loans': lambda x: max(item['n_loans'] for item in x),  # Maximum
        'amm_price': lambda x: sum(item['amm_price'] for item in x) / count,
        'price_oracle': lambda x: sum(item['price_oracle'] for item in x) / count,
        'base_price': lambda x: sum(item['base_price'] for item in x) / count,
        'min_band': lambda x: min(item['min_band'] for item in x),  # Minimum
        'max_band': lambda x: max(item['max_band'] for item in x),  # Maximum
        'borrowable': lambda x: sum(item['borrowable'] for item in x) / count,
        'loan_discount': lambda x: sum(item['loan_discount'] for item in x) / count,
        'liquidation_discount': lambda x: sum(item['liquidation_discount'] for item in x) / count,
        'sum_debt_squared': lambda x: sum(item['sum_debt_squared'] for item in x) / count
    }
    
    # Calculate aggregations
    result = {}
    for field, agg_func in aggregations.items():
        try:
            result[field] = agg_func(selected_items)
        except (KeyError, TypeError):
            continue  # Skip if field is missing or invalid
    
    # Add the time range for reference
    result['time_range'] = {
        'start': selected_items[0]['dt'],
        'end': selected_items[-1]['dt']
    }
    
    return result


def prices_get_crvusd_market_snapshots(address):
    url = f"/v1/crvusd/markets/ethereum/{address}/snapshots?fetch_on_chain=true&agg=day"
    data = prices_api_request(url)
    cur_snapshot = data['data'][0]
    prev_snapshot = data['data'][7]
    week_snapshot = aggregate_crvusd_snapshots(data['data'], 1, 7)
    prev_week_snapshot = aggregate_crvusd_snapshots(data['data'], 8, 15)
    return [cur_snapshot, week_snapshot, prev_snapshot, prev_week_snapshot]


def prices_get_lending_liquidations(chain, address, start, end):
    prev_start = start - (end - start)
    prev_end = start
    url = f"/v1/lending/liquidations/{chain}/{address}/history/detailed?start={prev_start}&end={end}"
    data = prices_api_request(url)
    liqs = 0
    prev_liqs = 0
    self_liqs = 0
    prev_self_liqs = 0
    for item in data['data']:
        ts = epoch_time_formatter(item['dt'])
        if ts >= start and ts <= end:
            liqs += 1
            if item['self'] == True:
                self_liqs += 1
        elif ts >= prev_start and ts <= prev_end:
            prev_liqs += 1
            if item['self'] == True:
                prev_self_liqs += 1

    return liqs, prev_liqs, self_liqs, prev_self_liqs

### Curve Frontend-JS API

In [8]:
def frontend_api_request(method):
    base_url = "https://api.curve.fi/v1"
    url = base_url + method
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to retrieve data. Status code: {response.status_code}")
    
def get_hidden_pools():
    method = "/getHiddenPools"
    data = frontend_api_request(method)
    hidden_pools = {}
    for chain in data['data']:
        hidden_pools[chain] = {}
        for pool in data['data'][chain]:
            hidden_pools[chain][pool] = True
    for pool in MY_HIDDEN_POOLS_FILTER:
        chain = MY_HIDDEN_POOLS_FILTER[pool]
        if chain not in hidden_pools:
            hidden_pools[chain] = {}
        hidden_pools[chain][pool] = True
    return hidden_pools

def get_all_pools(chains):
    all_pools = []
    for chain in chains:
        for pool_type in POOL_TYPES:
            method = f'/getPools/{chain}/{pool_type}'
            data = frontend_api_request(method)
            for pool in data['data']['poolData']:
                pool['blockchainId'] = chain
                pool['type'] = POOL_TYPES[pool_type]
                pool['registryId'] = pool_type
                all_pools.append(pool)
    return all_pools

def get_all_lending_markets():
    method = '/getLendingVaults/all'
    data = frontend_api_request(method)
    return data['data']['lendingVaultData'], data['data']['tvl']

def get_all_gauges():
    method = '/getAllGauges'
    data = frontend_api_request(method)
    gauge_data = {}
    for gauge in data['data']:
        gauge = data['data'][gauge]
        chain = gauge['blockchainId']
        if chain not in gauge_data:
            gauge_data[chain] = {}
        gauge_data[chain][gauge['gauge']] = gauge
    return gauge_data

test = get_all_gauges()

def get_volumes(chains):
    method = "/getVolumes/"
    volume_data = {}
    for chain in chains:

        chain_data = frontend_api_request(method + chain)
        volume_data[chain] = {}
        for item in chain_data['data']['pools']:
            address = item['address']
            volume_data[chain][address] = item

    return volume_data

### HTML Generation

In [9]:
def make_metric_html(name, value, prevValue, showPercentChange=True, valuePrefix='', valuePostfix='', customValueFormat=None, changeFormatting=",.2f", reversed=False):
    try:
        if reversed == False:
            if showPercentChange:
                if value > prevValue:
                    change = f"""<span class="positive-change">+{format((value - prevValue)/prevValue*100, changeFormatting)}%</span>"""
                else:
                    change = f"""<span class="negative-change">{format((value - prevValue)/prevValue*100, changeFormatting)}%</span>"""
            else:
                if value > prevValue:
                    change = f"""<span class="positive-change">+{valuePrefix}{format(value - prevValue, changeFormatting)}{valuePostfix}</span>"""
                else:
                    change = f"""<span class="negative-change">-{valuePrefix}{format(prevValue - value, changeFormatting)}{valuePostfix}</span>"""
        else:
            if showPercentChange:
                if value > prevValue:
                    change = f"""<span class="negative-change">+{format((value - prevValue)/prevValue*100, changeFormatting)}%</span>"""
                else:
                    change = f"""<span class="positive-change">{format((value - prevValue)/prevValue*100, changeFormatting)}%</span>"""
            else:
                if value > prevValue:
                    change = f"""<span class="negative-change">+{valuePrefix}{format(value - prevValue, changeFormatting)}{valuePostfix}</span>"""
                else:
                    change = f"""<span class="positive-change">-{valuePrefix}{format(prevValue - value, changeFormatting)}{valuePostfix}</span>"""

        if customValueFormat:
            value = customValueFormat
        else:
            value = f"${format_number(value, 4, 3)}"

        return {'Metric': name, 'Value': value, 'Change': change}
    except:
        pass


def make_chain_with_icon_html(chain):
    return f"""<div class="network-cell"><img title={chain.capitalize()} src='https://raw.githubusercontent.com/curvefi/curve-assets/refs/heads/main/chains/{chain}.png' class="network-icon"></div>"""


def make_name_link_html(name, url=None, img_link=None):
    if img_link:
        return f"""<a href='{url}'><div class="network-cell">{img_link}<span>{name}</span><</div></a>"""
    if url:
        return f"""<a href='{url}'><div class="pool-content">{name}</div></a>"""
    else:
        return f"""<div class="pool-content">{name}</div>"""


def get_coin_image_url(address, chain, url_dict):
    if chain == 'ethereum':
        base_image_url = 'https://raw.githubusercontent.com/curvefi/curve-assets/refs/heads/main/images/assets/'
    else:
        base_image_url = f'https://raw.githubusercontent.com/curvefi/curve-assets/refs/heads/main/images/assets-{chain}/'

    coin_image_url = f'{base_image_url}{address.lower()}.png'
    if (chain in url_dict and address in url_dict[chain] and url_dict[chain][address] != "") or url_exists(coin_image_url):
        if chain not in url_dict:
            url_dict[chain] = {}
        image_link = f"""<img src='{coin_image_url}' class="token-icon">"""
    else:
        image_link = ""
    url_dict[chain][address] = image_link
    return image_link, url_dict


def get_coin_html(coin, chain, url_dict, market_type):
    image_link, url_dict = get_coin_image_url(coin['address'], chain, url_dict)
    coin_pill = f"""<div class="token-pill-{market_type}">{image_link}<span>{coin['symbol']}</span></div>"""
    return coin_pill, url_dict


def df_to_html(df, classes='curve-table'):

    html_str = "<div class='curve-table-container'>"           
    html_str += df.to_html( 
        index=False, 
        classes=classes,
        border=0,
        justify='left',
        escape=False
    )
    html_str += "</div>"
    return html_str


def make_metric_name_with_images(content):
    html_str = f"""<div class="metrics-name-cell">"""
    for item in content:
        if item['type'] == 'img_html':
            html_str += item['content']
        elif item['type'] == 'text':
            html_str += f"""<span>{item['content']}</span>"""
    html_str += "</div>"
    return html_str


def display_curve_table(df_html, html_styling):
    
    # Add necessary classes to the table
    df_html = df_html.replace('<table', '<div class="curve-table-container"><table class="curve-table"')
    df_html = df_html.replace('</table>', '</table></div>')
    
    # Wrap everything in a gh-content container for proper styling
    final_html = f"""
    <div class="gh-content gh-canvas">
        {html_styling}
        {df_html}
    </div>
    """
    
    return HTML(final_html)

### Dataframe Processing

In [10]:
def get_top_yielding_lending_markets(base_df, print_num, minSupplied=0, normalize=True, columns_to_show=['Chain', 'Market', 'Yield']):
    df = base_df.copy()

    if len(df) == 0:
        return pd.DataFrame(columns=columns_to_show)

    if normalize:
        df['yieldMin'] = np.where(df['suppliedUsd'] < minSupplied,
                                df['borrowApy']*(df['borrowedUsd'] / minSupplied) + df['rewardsMinApy']*(df['suppliedUsd'] / minSupplied),
                                df['borrowApy']*(df['borrowedUsd'] / df['suppliedUsd']) + df['rewardsMinApy']
        )

        df['yieldMax'] = np.where(df['suppliedUsd'] < minSupplied,
                                df['borrowApy']*(df['borrowedUsd'] / minSupplied) + df['rewardsMaxApy']*(df['suppliedUsd'] / minSupplied),
                                df['borrowApy']*(df['borrowedUsd'] / df['suppliedUsd']) + df['rewardsMaxApy']
        )
    else:
        df = df[df['suppliedUsd'] >= minSupplied]
        df['yieldMin'] = df['lendApy'] + df['rewardsMinApy']
        df['yieldMax'] = df['lendApy'] + df['rewardsMaxApy']
    
    df = df.sort_values('yieldMin', ascending=False).head(print_num)
    df['yieldStr'] = df['yieldMin'].apply(lambda x: format_number(x)) + '%'

    df = df.rename(columns={'nameLink': 'Market',
                                    'chainWithIcon': 'Chain',
                                    'yieldStr': 'Yield'})  
    
    return df[columns_to_show]


def get_top_pool_yields_pools(base_df, print_num, minTvl=0, normalize=True, asset=None, columns_to_show=['Chain', 'Pool', 'Type', 'Yield']):
    df = base_df.copy()
    if asset:
        df = df[df['assetType'] == asset]
        df['assetType'] = df['assetType'].str.upper()

    if len(df) == 0:
        return pd.DataFrame(columns=columns_to_show)

    if normalize == True:
        df['yieldMin'] = np.where(df['tvlUsd'] < minTvl,
                                df['lstApy'] + (df['tradingApy'] + df['rewardsMinApy']) * (df['tvlUsd'] / (minTvl)),
                                df['lstApy'] + df['tradingApy'] + df['rewardsMinApy'])

        df['yieldMax'] = np.where(df['tvlUsd'] < minTvl,
                                df['lstApy'] + (df['tradingApy'] + df['rewardsMaxApy']) * (df['tvlUsd'] / (minTvl)),
                                df['lstApy'] + df['tradingApy'] + df['rewardsMaxApy'])
        df['tvlUsd'] = np.where(df['tvlUsd'] < minTvl, (minTvl), df['tvlUsd'])
        df['tvlUsd'] = df['tvlUsd'].map('${:,.0f}'.format)
    else:
        df = df[df['tvlUsd'] > minTvl]
        df['yieldMin'] = df['lstApy'] + df['tradingApy'] + df['rewardsMinApy']
        df['yieldMax'] = df['lstApy'] + df['tradingApy'] + df['rewardsMaxApy']
        df['tvlUsd'] = df['tvlUsd'].map('${:,.0f}'.format)

    df = df.sort_values('yieldMin', ascending=False)
    df['yieldStr'] = df['yieldMin'].apply(lambda x: format_number(x)) + '%'
    df = df.rename(columns={'nameLink': 'Pool',
                                    'chainWithIcon': 'Chain',
                                    'tvlUsd': 'TVL',
                                    'yieldStr': 'Yield',
                                    'assetType': 'Type'})  
    
    return df[columns_to_show].head(print_num)


def sort_pool_df_to_html(base_df, item_num, sort_column, columns_to_show, ascending=False):
    df = base_df.copy()
    sorted_df = df.sort_values(sort_column, ascending=ascending).head(item_num)
    sorted_df['liqUtilizationDaily'] = sorted_df['liqUtilizationDaily'].map('{:,.2f}%'.format)
    sorted_df['feesUsd'] = sorted_df['feesUsd'].apply(lambda x: f"${format_number(x, 4, 3)}")
    sorted_df['volumeUsd'] = sorted_df['volumeUsd'].apply(lambda x: f"${format_number(x, 4, 3)}")
    sorted_df['tvlUsd'] = sorted_df['tvlUsd'].apply(lambda x: f"${format_number(x, 4, 3)}")
    sorted_df['baseWeeklyApy'] = sorted_df['baseWeeklyApy'].map('{:,.2f}%'.format)
    column_name_list = {'nameLink': 'Pool',
                                'chainWithIcon': 'Chain',
                                'tvlUsd': 'TVL',
                                'yieldStr': 'Yield',
                                'assetType': 'Type',
                                'volumeUsd': 'Volume',
                                'feesUsd': 'Fees',}
    sorted_df = sorted_df.rename(columns=column_name_list)
    renamed_column_list = [column_name_list[column] for column in columns_to_show]

    return df_to_html(sorted_df[renamed_column_list], classes='curve-table chain-column pool-column')

# Data Collection

### Get Times

In [46]:
# Get the times required for the weekly thursday - thursday period
now = datetime.now(pytz.utc)
recent_thursday = now - timedelta(days=(now.weekday() + 4) % 7, hours=now.hour, minutes=now.minute, seconds=now.second, microseconds=now.microsecond)
previous_thursday = recent_thursday - timedelta(days=7)
epoch_start = int(previous_thursday.timestamp())
epoch_end = int(recent_thursday.timestamp())

### Load asset image dictionary (if available)

In [47]:
try:
    with open('curve-assets-image-html.dict', 'r') as file:
        coin_image_html_dict = literal_eval(file.read())
except FileNotFoundError:
    print("curve-assets-image-html.dict not found, this run will generate this file, but will take a little longer to run.")
    coin_image_html_dict = {} 

### Get Basic Data from Curve Prices API

In [13]:
chains = get_all_chains()
prices_chain_transactions = get_chain_tx_count(epoch_start, epoch_end)
prices_chain_users = get_chain_user_stats(epoch_start, epoch_end)
prices_pools = get_all_chain_pools(chains)
prices_lending_markets = get_all_chain_lending_markets(chains)
prices_crvusd_markets = get_crvusd_markets()

### Get Basic Data From Curve Frontend-JS API

In [14]:
hidden_pools = get_hidden_pools()
fe_pool_data_raw = get_all_pools(chains)
volume_data = get_volumes(chains)
fe_lending_data_raw, lending_tvl = get_all_lending_markets()
fe_gauge_data = get_all_gauges()

# Specific Market Data Acquisition & Processing

## Lending Markets

In [24]:
# Processes lending market data and grabs historical data, takes around 1-3mins to run

fe_lending_data = fe_lending_data_raw.copy()
kill_flag = False


market_num = 0
for market in fe_lending_data:

    if kill_flag:
        break
    market_num += 1
    print(f"Processing market {market_num} of {len(fe_lending_data)}")

    chain = market['blockchainId']
    address = market['address']
    gauge = None

    if 'gaugeAddress' in market and market['gaugeAddress'] in fe_gauge_data:
        gauge = fe_gauge_data[market['gaugeAddress']]

    # make hyperlink for market
    coin_collateral, coin_image_html_dict = get_coin_html(market['assets']['collateral'], chain, coin_image_html_dict, 'pool')
    coin_borrow, coin_image_html_dict = get_coin_html(market['assets']['borrowed'], chain, coin_image_html_dict, 'borrow')
    market_str = f"""{coin_borrow}{coin_collateral}"""
    try: 
        market['nameLink'] = make_name_link_html(market_str, market['lendingVaultUrls']['deposit'])
    except:
        market['nameLink'] = make_name_link_html(market_str)

    # make network with icon
    market['chainWithIcon'] = make_chain_with_icon_html(chain)

    # get historical data for each market
    try:
        prev_snapshot, cur_snapshot = prices_get_lending_snapshots(chain, market['controllerAddress'], epoch_start, epoch_end)
        market['liqs'], market['prev_liqs'], market['self_liqs'], market['prev_self_liqs'] = prices_get_lending_liquidations(chain, market['controllerAddress'], epoch_start, epoch_end)
        market['prevUtilization'] = 0
        if prev_snapshot['total_debt_usd'] != 0 and prev_snapshot['total_assets_usd'] != 0:
            market['prevUtilization'] = prev_snapshot['total_debt_usd'] / prev_snapshot['total_assets_usd'] * 100
        
        market['prevBorrowApy'] = prev_snapshot['borrow_apy']
        market['prevLendApy'] = prev_snapshot['lend_apy']
        market['prevBorrowedUsd'] = prev_snapshot['total_debt_usd']
        market['prevSuppliedUsd'] = prev_snapshot['total_assets_usd']
        market['prevCollateralUsd'] = prev_snapshot['collateral_balance_usd']
        market['prevTvlUsd'] = prev_snapshot['total_assets_usd'] - prev_snapshot['total_debt_usd'] + prev_snapshot['collateral_balance_usd']
        market['prevNumLoans'] = prev_snapshot['n_loans']
        market['numLoans'] = cur_snapshot['n_loans']
        market['prevSnapshot'] = prev_snapshot
        market['curSnapshot'] = cur_snapshot
    except:
        market['prevUtilization'] = 0
        market['prevBorrowApy'] = 0
        market['prevLendApy'] = 0
        market['prevBorrowedUsd'] = 0
        market['prevSuppliedUsd'] = 0
        market['prevSnapshot'] = None
        market['tvlUsd'] = 0
        market['prevCollateralUsd'] = 0
        market['prevTvlUsd'] = 0

    # Make various stats for the market
    market['utilization'] = 0
    if market['borrowed']['usdTotal'] != 0 and market['totalSupplied']['usdTotal'] != 0:
        market['utilization'] = market['borrowed']['usdTotal'] / market['totalSupplied']['usdTotal'] * 100
    
    market['borrowApy'] = market['rates']['borrowApyPcent']
    market['lendApy'] = market['rates']['lendApyPcent']
    market['borrowedUsd'] = market['borrowed']['usdTotal']
    market['suppliedUsd'] = market['totalSupplied']['usdTotal']
    if 'ammBalances' in market \
            and 'ammBalanceCollateralUsd' in market['ammBalances'] \
            and 'ammBalanceBorrowedUsd' in market['ammBalances'] \
            and market['ammBalances']['ammBalanceBorrowedUsd'] != None \
            and market['ammBalances']['ammBalanceCollateralUsd'] != None:
        market['collateralUsd'] = market['ammBalances']['ammBalanceBorrowedUsd'] + market['ammBalances']['ammBalanceCollateralUsd']
    else:
        market['collateralUsd'] = 0
    market['tvlUsd'] = market['suppliedUsd'] - market['borrowedUsd'] + market['collateralUsd']

    # Make rewards APYs for the market
    market['rewardsMinApy'] = 0
    market['rewardsMaxApy'] = 0
    if gauge != None and 'gaugeCrvApy' in gauge and gauge['gaugeCrvApy'] != [] and gauge['gaugeCrvApy'] != [None, None]:
        market['rewardsMinApy'] += gauge['gaugeCrvApy'][0]
        market['rewardsMaxApy'] += gauge['gaugeCrvApy'][1]

    if 'gaugeRewards' in market and market['gaugeRewards'] != []:
        for reward in market['gaugeRewards']:
            market['rewardsMinApy'] += reward['apy']
            market['rewardsMaxApy'] += reward['apy']

Processing market 1 of 59
Processing market 2 of 59
Processing market 3 of 59
Processing market 4 of 59
Processing market 5 of 59
Processing market 6 of 59
Processing market 7 of 59
Processing market 8 of 59
Processing market 9 of 59
Processing market 10 of 59
Processing market 11 of 59
Processing market 12 of 59
Processing market 13 of 59
Processing market 14 of 59
Processing market 15 of 59
Processing market 16 of 59
Processing market 17 of 59
Processing market 18 of 59
Processing market 19 of 59
Processing market 20 of 59
Processing market 21 of 59
Processing market 22 of 59
Processing market 23 of 59
Processing market 24 of 59
Processing market 25 of 59
Processing market 26 of 59
Processing market 27 of 59
Processing market 28 of 59
Processing market 29 of 59
Processing market 30 of 59
Processing market 31 of 59
Processing market 32 of 59
Processing market 33 of 59
Processing market 34 of 59
Processing market 35 of 59
Processing market 36 of 59
Processing market 37 of 59
Processing

In [25]:
lending_df = pd.DataFrame(fe_lending_data)

# Not currently using top lending market yields in report
lending_top_yields_df = get_top_yielding_lending_markets(lending_df, 5, 10000, normalize=True)

# Make the metrics for the lending markets
metrics_list = []

metrics_list.append(make_metric_html('Lending TVL', lending_df['tvlUsd'].sum(), lending_df['prevTvlUsd'].sum()))
metrics_list.append(make_metric_html('Supplied', lending_df['suppliedUsd'].sum(), lending_df['prevSuppliedUsd'].sum()))
metrics_list.append(make_metric_html('Borrowed', lending_df['borrowedUsd'].sum(), lending_df['prevBorrowedUsd'].sum()))
metrics_list.append(make_metric_html('Collateral', lending_df['collateralUsd'].sum(), lending_df['prevCollateralUsd'].sum()))
metrics_list.append(make_metric_html(
        'Loans', 
        int((lending_df['numLoans']).sum()), 
        int((lending_df['prevNumLoans']).sum()), 
        showPercentChange=False, 
        changeFormatting=",.0f",
        customValueFormat=str(int((lending_df['numLoans']).sum()))
    ))

lending_metrics_html = df_to_html(pd.DataFrame(metrics_list), classes='curve-table')

## Pools

#### Data Acquisition

In [26]:
# This has to do a few calls to different APIs for each pool, so it takes a while to run, around 30-90mins
# To speed it up, increase the TVL threshold so it only processes pools with a TVL above that threshold
TVL_THRESHOLD = 100

In [27]:
fe_pool_data = fe_pool_data_raw.copy()

# shuffle the list so pool estimated time remaining is more accurate
random.shuffle(fe_pool_data)

num_pools = len(fe_pool_data)
i_pool = 0
kill_flag = False
good_pool_data = []
time_start = int(time.time())

for pool in fe_pool_data:
    if kill_flag == True:
        break

    chain = pool['blockchainId']
    address = pool['address']
    i_pool += 1

    time_elapsed = int(time.time()) - time_start
    time_per_item = time_elapsed / i_pool
    est_time_remaining = int((num_pools - i_pool) * time_per_item)

    if (chain in hidden_pools and pool['id'] in hidden_pools[chain]) \
            or (address in MY_HIDDEN_POOLS_FILTER and MY_HIDDEN_POOLS_FILTER[address] == chain) \
            or pool['isBroken'] == True \
            or chain not in volume_data \
            or chain not in chains \
            or pool['usdTotal'] < TVL_THRESHOLD:
        print(f"Skipping pool {pool['name']} ({i_pool}/{num_pools}), {chain}-{address}, TVL: {pool['usdTotal']}, est. time remaining: {est_time_remaining//60}m{est_time_remaining%60}s")
        continue
    
    pool_volume = volume_data[chain][address]
    
    # get volume data and fees from prices.curve.fi if available
    pool['volumeUsd'] = 0
    pool['feesUsd'] = 0
    pool['prevVolumeUsd'] = 0
    pool['prevFeesUsd'] = 0
    pool['prevTvlUsd'] = 0
    pool['prevTokenSupply'] = 0
    pool['tokenSupply'] = int(pool['totalSupply'])/10**18
    pool['baseWeeklyApy'] = pool_volume['latestWeeklyApyPcent']

    print(f"Processing {pool['name']} ({i_pool}/{num_pools}), {chain}-{address}, est. time remaining: {est_time_remaining//60}m{est_time_remaining%60}s")
    try:
        pool['volumeUsd'], pool['feesUsd'], pool['prevVolumeUsd'], pool['prevFeesUsd'] = get_pool_volume(chain, address, epoch_start, epoch_end)
    except:
        print(f"Failed getting volume data for pool {pool['name']} ({i_pool}/{num_pools}): apr {pool['baseWeeklyApy']}, tvl {pool['usdTotal']}, {chain}-{address}")
        continue
    try:
        tvl_data = get_tvl(chain, address, pool['creationTs'], epoch_start, get_cur_time())
        pool['prevTvlUsd'] = tvl_data['start']['tvl']
        pool['prevTokenSupply'] = tvl_data['start']['token_supply']
        pool['tvlData'] = tvl_data
        if tvl_data['end']['token_supply_snapshot'] != None:
            pool['tvlUsd'] = tvl_data['end']['tvl']
            pool['tokenSupply'] = tvl_data['end']['token_supply']
    except:
        print(f"Failed getting tvl data for pool {pool['name']} ({i_pool}/{num_pools}): apr {pool['baseWeeklyApy']}, tvl {pool['usdTotal']}, {chain}-{address}")
        continue
    
    good_pool_data.append(pool)


Skipping pool KNOX/eUSD (1/3794), arbitrum-0x96e5F3F6e055ecc4C84302f3AD5295c47cD59914, TVL: 0, est. time remaining: 0m0s
Skipping pool BTC (2/3794), fantom-0x6A1C781B7B280E3c8BF04FDfb86C112C9Ac70a89, TVL: 0, est. time remaining: 0m0s
Skipping pool wrapped Bitcoin Pax Gold (3/3794), polygon-0xF2ce93d54e0b043fbe654eA73aAde9DFB092d2cc, TVL: 0, est. time remaining: 0m0s
Skipping pool AMPH/WETH/wUSDA (4/3794), ethereum-0x05CA1ff6fF45e55906c86Ad0d3FB2EbFaE9E0891, TVL: 0, est. time remaining: 0m0s
Processing frxETH/afETH (5/3794), ethereum-0x55Faa4a58f6510f78c1466C71C63200D62a197D2, est. time remaining: 0m0s
Processing SETH/ETH (6/3794), arbitrum-0x390C6eB1E824A7CCa2E646e47D8fCba4263E9E38, est. time remaining: 42m5s
Processing crvUSD/cncCRVUSD (7/3794), ethereum-0xe3E637f8aCC097244A065791142c29fbF5877D18, est. time remaining: 99m11s
Skipping pool USD0/USDC (8/3794), ethereum-0x7FE188ACa91077353c9Edb0E9e4f34B60210711f, TVL: 0, est. time remaining: 110m25s
Processing Curve.fi Factory Crypto Poo

#### Optional, save/load pulled data as csv files

In [28]:
# SAVE

with open(f'good-pool-data-{epoch_start}-{epoch_end}.txt', 'w') as file:
    file.write(str(good_pool_data))

formatted_pool_data = good_pool_data.copy()

In [34]:
# LOAD

with open(f'good-pool-data-{epoch_start}-{epoch_end}.txt', 'r') as file:
    formatted_pool_data = literal_eval(file.read())

good_pool_data = formatted_pool_data.copy()

#### Data Processing

In [29]:
# Separated data acquisition and processing to make it easier to re-run the processing step
# This processing step may take a few minutes to run the first time, as it has to create a dict of coin image URLs if they exist

# Copy the pool data so we still have the original data
formatted_pool_data = good_pool_data.copy()

# Process each pool in the data set
i = 0
for pool in formatted_pool_data:
    i += 1
    print(f"Processing pool {i} of {len(formatted_pool_data)}")
    
    chain = pool['blockchainId']
    address = pool['address']

    pool_volume = volume_data[chain][address]
    
    pool['assetType'] = 'other'
    coin_str = ""
    pool['crvusd_pool'] = False

    for coin in pool['coins']:
        coin_symbol = coin['symbol'].lower()
        
        # see if it's a crvusd pool
        if (coin_symbol == 'crvusd' or coin_symbol == 'scrvusd') and pool['type'] == 'stableswap':
            pool['crvusd_pool'] = True

        # see if it's a stableswap pool with a known asset type
        if coin_symbol in POOL_TYPES_ASSET_DICT and pool['type'] == 'stableswap':
            pool['assetType'] = POOL_TYPES_ASSET_DICT[coin_symbol]
        
        # see if it's a cryptoswap pool with crv
        elif coin_symbol in POOL_TYPES_ASSET_DICT and pool['type'] == 'cryptoswap' and POOL_TYPES_ASSET_DICT[coin_symbol] == 'crv':
            pool['assetType'] = 'crv'

        coin_image_html, coin_image_html_dict = get_coin_html(coin, chain, coin_image_html_dict, "pool")
        coin_str += coin_image_html

    # If there's more than 4 coins in the pool, just show the pool name
    if len(pool['coins']) > 4:
        coin_str = pool['name']
    
    # make the pool name link
    try:
        # if the chain is xdai, the link to the frontend should actually be the second link in the array, first link is broken
        if chain == 'xdai':
            pool['nameLink'] = make_name_link_html(coin_str, pool['poolUrls']['deposit'][1])
        else:
            pool['nameLink'] = make_name_link_html(coin_str, pool['poolUrls']['deposit'][0])
    except:
        # if there's no deposit link, just show the pool name
        pool['nameLink'] = make_name_link_html(coin_str)

    # make network with icon
    pool['chainWithIcon'] = make_chain_with_icon_html(chain)

    # calculate APYs and stats
    pool['lstApy'] = pool_volume['includedApyPcentFromLsts']
    pool['tradingApy'] = pool_volume['latestWeeklyApyPcent'] - pool['lstApy']
    pool['baseWeeklyApy'] = pool_volume['latestWeeklyApyPcent']
    pool['liqUtilizationDaily'] = 0
    pool['liqUtilizationDaily'] = pool['volumeUsd'] / pool['tvlUsd'] * 100 / 7
    pool['prevLiqUtilizationDaily'] = 0
    pool['newPool'] = False

    # If we have no previous TVL data, assume it's a new pool
    if pool['prevTvlUsd'] != 0:
        pool['prevLiqUtilizationDaily'] = pool['prevVolumeUsd'] / pool['prevTvlUsd'] * 100 / 7
    else:
        pool['newPool'] = True

    # make rewards APYs for the pool
    pool['rewardsMinApy'] = 0
    pool['rewardsMaxApy'] = 0
    if 'gaugeCrvApy' in pool and pool['gaugeCrvApy'] != [] and pool['gaugeCrvApy'] != [None, None]:
        pool['rewardsMinApy'] += pool['gaugeCrvApy'][0]
        pool['rewardsMaxApy'] += pool['gaugeCrvApy'][1]

    if 'gaugeRewards' in pool and pool['gaugeRewards'] != []:
        for reward in pool['gaugeRewards']:
            pool['rewardsMinApy'] += reward['apy']
            pool['rewardsMaxApy'] += reward['apy']

    pool['rewardYieldStr'] =  f"{pool['rewardsMinApy']:,.2f}-{pool['rewardsMaxApy']:,.2f}%"
    
# Make the df from the pool data
pools_df = pd.DataFrame(formatted_pool_data)

# Save/update the curve asset image html dict
with open('curve-assets-image-html.dict', 'w') as file:
    file.write(str(coin_image_html_dict))

Processing pool 1 of 814
Processing pool 2 of 814
Processing pool 3 of 814
Processing pool 4 of 814
Processing pool 5 of 814
Processing pool 6 of 814
Processing pool 7 of 814
Processing pool 8 of 814
Processing pool 9 of 814
Processing pool 10 of 814
Processing pool 11 of 814
Processing pool 12 of 814
Processing pool 13 of 814
Processing pool 14 of 814
Processing pool 15 of 814
Processing pool 16 of 814
Processing pool 17 of 814
Processing pool 18 of 814
Processing pool 19 of 814
Processing pool 20 of 814
Processing pool 21 of 814
Processing pool 22 of 814
Processing pool 23 of 814
Processing pool 24 of 814
Processing pool 25 of 814
Processing pool 26 of 814
Processing pool 27 of 814
Processing pool 28 of 814
Processing pool 29 of 814
Processing pool 30 of 814
Processing pool 31 of 814
Processing pool 32 of 814
Processing pool 33 of 814
Processing pool 34 of 814
Processing pool 35 of 814
Processing pool 36 of 814
Processing pool 37 of 814
Processing pool 38 of 814
Processing pool 39 of

In [ ]:
# Sometimes bugged pools will show up in the lists, use this list to filter them at this step
# Set Test_Pools to show extra pools and filter out pools in the filter list
TEST_POOLS = False
POOL_FILTER_LIST = [
    ('fantom', '0x3f833Ed02629545DD78AFc3D585f7F3918a3De62'),
    ('ethereum', '0xC03FEF1c425956A3Cd5762022E511e0d4148B3D6'),
    ('fraxtal', '0xeE454138083b9B9714cac3c7cF12560248d76D6B'),
    ('fraxtal', '0x4Cfc391d75c43Cf1Bdb368e8bF680AEd1228df39'),
    ('ethereum', '0xFE3C78D947b329160496E192b4Cf417bB86272Ed'),
    ('arbitrum', '0xad37295881f53Fe6FDAb54493A06CD84f988646B'),
    ('arbitrum', '0x845C8bc94610807fCbaB5dd2bc7aC9DAbaFf3c55'),
    ('optimism', '0xd8dD9a8b2AcA88E68c46aF9008259d0EC04b7751'),
]

df = pools_df.copy()

# take out pools with base yields greater than 1000% because these will be errors or depegged pools
df = df[df['baseWeeklyApy'] < 1000]

# take out any pools in the filter list
df = df[~df.apply(lambda x: (x['blockchainId'], x['address']) in POOL_FILTER_LIST, axis=1)]

if TEST_POOLS:
    df_test = df.copy()
    df_test['chainAddress'] = df_test.apply(lambda row: f"('{row['blockchainId']}', '{row['address']}')", axis=1)
    df_usd = get_top_pool_yields_pools(df_test, 10, 100000, asset='usd', columns_to_show=['Chain', 'Pool', 'Type', 'Yield', 'chainAddress'])
    df_btc = get_top_pool_yields_pools(df_test, 6, 100000, asset='btc', columns_to_show=['Chain', 'Pool', 'Type', 'Yield', 'chainAddress'])
    df_eth = get_top_pool_yields_pools(df_test, 6, 100000, asset='eth', columns_to_show=['Chain', 'Pool', 'Type', 'Yield', 'chainAddress'])
    df_crv = get_top_pool_yields_pools(df_test, 6, 100000, asset='crv', columns_to_show=['Chain', 'Pool', 'Type', 'Yield', 'chainAddress'])
    df_eur = get_top_pool_yields_pools(df_test, 3, 100000, asset='eur', columns_to_show=['Chain', 'Pool', 'Type', 'Yield', 'chainAddress'])
else:
    df_usd = get_top_pool_yields_pools(df, 3, 100000, asset='usd')
    df_btc = get_top_pool_yields_pools(df, 2, 100000, asset='btc')
    df_eth = get_top_pool_yields_pools(df, 2, 100000, asset='eth')
    df_crv = get_top_pool_yields_pools(df, 2, 100000, asset='crv')
    df_eur = get_top_pool_yields_pools(df, 1, 100000, asset='eur')

top_pool_yields_html = df_to_html(pd.concat([df_usd, df_btc, df_eth, df_crv, df_eur], axis=0), classes='curve-table chain-column pool-column')

display_curve_table(top_pool_yields_html, HTML_STYLING)

Chain,Pool,Type,Yield
,CrossCurve Stable 2,USD,101.5%
,USD3scrvUSD,USD,50.57%
,USD3sDAI,USD,28.92%
,zunBTCtBTC,BTC,134.5%
,CrossCurve BTC,BTC,34.51%
,CrossCurve ETH,ETH,59.89%
,frxETHOETH,ETH,34.49%
,CRVcrvUSD,CRV,33.79%
,CRVvsdCRVasdCRV,CRV,22.19%
,EURAEURTEURS,EUR,18.97%


### Get Top crvUSD & scrvUSD Yields

In [34]:
# COMBINE POOL AND LENDING YIELDS FOR CRVUSD YIELDS

top_crvusd_pool_yields_df = get_top_pool_yields_pools(df[df['crvusd_pool'] == True], 4, 100000, columns_to_show=['Chain', 'Pool', 'Yield', 'yieldMin'])
top_crvusd_pool_yields_df['Type'] = 'Pool'
crvusd_pools_df = top_crvusd_pool_yields_df.rename(columns={'Pool': 'Market'})
top_lending_yields_df = get_top_yielding_lending_markets(lending_df, 4, 10000, normalize=True, columns_to_show=['Chain', 'Market', 'Yield', 'yieldMin'])
top_lending_yields_df['Type'] = 'LlamaLend'
crvusd_yield_df = pd.concat([crvusd_pools_df, top_lending_yields_df], axis=0).sort_values('yieldMin', ascending=False).head(8)
top_crvusd_yields_html = df_to_html(crvusd_yield_df[['Chain', 'Market', 'Type', 'Yield']], classes='curve-table chain-column pool-column')

display_curve_table(top_crvusd_yields_html, HTML_STYLING)

Chain,Market,Type,Yield
,USD3scrvUSD,Pool,50.57%
,crvUSDwstETH,LlamaLend,40.57%
,crvUSDUwU,LlamaLend,32.78%
,crvUSDWETH,LlamaLend,32.67%
,crvUSDsFRAX,LlamaLend,28.39%
,USDCscrvUSD,Pool,27.62%
,TUSDcrvUSD,Pool,27.17%
,crvUSDzunUSD,Pool,26.97%


In [35]:
# Highest Pools Volume
columns_to_show = ['chainWithIcon', 'nameLink', 'volumeUsd', 'tvlUsd']
pool_highest_volume_html = sort_pool_df_to_html(df, 4, 'volumeUsd', columns_to_show)

display_curve_table(pool_highest_volume_html, HTML_STYLING)

Chain,Pool,Volume,TVL
,DAIUSDCUSDT,$624.1M,$168.7M
,ETHstETH,$579.4M,$181.4M
,USD0USDC,$246.8M,$76.18M
,sDAIsUSDe,$197.8M,$41.71M


In [36]:
#biggest new pools of the week
columns_to_show = ['chainWithIcon', 'nameLink', 'tvlUsd']
pool_biggest_new_pools_html = sort_pool_df_to_html(df[df['newPool']==True], 4, 'tvlUsd', columns_to_show)

display_curve_table(pool_biggest_new_pools_html, HTML_STYLING)

Chain,Pool,TVL
,USDCUSDtb,$19.98M
,USDasUSDa,$5.037M
,USDeUSDtb,$164.3k
,zunBTCtBTC,$140.8k


In [37]:
# Biggest Pools for fees
columns_to_show = ['chainWithIcon', 'nameLink', 'feesUsd', 'volumeUsd']
pool_biggest_fees_html = sort_pool_df_to_html(df, 3, 'feesUsd', columns_to_show)

display_curve_table(pool_biggest_fees_html, HTML_STYLING)

Chain,Pool,Fees,Volume
,DAIUSDCUSDT,$62.4k,$624.1M
,ETHstETH,$57.96k,$579.4M
,WETHCVX,$55.43k,$15.04M


In [38]:
# Most profitable pools
columns_to_show = ['chainWithIcon', 'nameLink', 'feesUsd', 'tvlUsd']
df['profitability'] = df['feesUsd'] / df['tvlUsd']
pool_most_profitable_html = sort_pool_df_to_html(df[df['tvlUsd']>100000], 3, 'profitability', columns_to_show)

display_curve_table(pool_most_profitable_html, HTML_STYLING)

Chain,Pool,Fees,TVL
,WETHCVX,$55.43k,$12.03M
,ETHXYO,$423.6,$107.7k
,CRVcrvUSDBTCETH,$9.488k,$2.459M


In [39]:
# Need to hardcode the TVL currently, for reasonsss...
pools_tvl = 2166000000
pools_tvl_prev = 2148000000

In [40]:
# Pools Metrics

pools_transactions_total = prices_chain_transactions['all']['pools']['cur_week_total']
pools_transactions_total_prev = prices_chain_transactions['all']['pools']['prev_week_total']
pools_transaction_ethereum = prices_chain_transactions['ethereum']['pools']['cur_week_total']
pools_transactions_ethereum_prev = prices_chain_transactions['ethereum']['pools']['prev_week_total']

pools_transactions_other = 0
pools_transactions_other_prev = 0
for chain in prices_chain_transactions:
    if chain != 'all' and chain != 'ethereum':
        pools_transactions_other += prices_chain_transactions[chain]['pools']['cur_week_total']
        pools_transactions_other_prev += prices_chain_transactions[chain]['pools']['prev_week_total']

pools_tranactions_per_sec = pools_transactions_total / 604800
pools_tranactions_per_sec_prev = pools_transactions_total_prev / 604800
pools_transactions_l2_ratio = pools_transactions_other / pools_transactions_total * 100
pools_transactions_l2_ratio_prev = pools_transactions_other_prev / pools_transactions_total_prev * 100

pools_fees_weekly = df['feesUsd'].sum()
pools_fees_weekly_prev = df['prevFeesUsd'].sum()

# Make the metrics list
metrics_list = []

metrics_list.append(make_metric_html('TVL', pools_tvl, pools_tvl_prev))
metrics_list.append(make_metric_html('Volume', df['volumeUsd'].sum(), df['prevVolumeUsd'].sum()))
metrics_list.append(make_metric_html('Transactions', pools_transactions_total, pools_transactions_total_prev, customValueFormat=f"{int(pools_transactions_total):,.0f}"))
metrics_list.append(make_metric_html('Total Fees', pools_fees_weekly, pools_fees_weekly_prev)) 
metrics_list.append(make_metric_html('DAO Fees', pools_fees_weekly/2, pools_fees_weekly_prev/2))

pool_metrics_html = df_to_html(pd.DataFrame(metrics_list), classes='curve-table')

display_curve_table(pool_metrics_html, HTML_STYLING)

Metric,Value,Change
TVL,$2.166B,+0.84%
Volume,$3.031B,+14.28%
Transactions,"387,579",-1.24%
Total Fees,$691.8k,-11.50%
DAO Fees,$345.9k,-11.50%


## crvUSD market data

In [41]:
def prices_get_crvusd_markets(coin_image_html_dict):
    url = "/v1/crvusd/markets/ethereum"
    data = prices_api_request(url)

    crvusd_markets = {}
    for contract in data['data']:
        address = contract['address']
        market_data = prices_get_crvusd_market_snapshots(address)
        contract['rate'] = market_data[1]['rate']
        contract['total_debt'] = market_data[0]['total_debt']
        contract['collateral_amount_usd'] = market_data[0]['total_collateral_usd']
        contract['n_loans'] = market_data[0]['n_loans']
        contract['prev_rate'] = market_data[3]['rate']
        contract['prev_collateral_amount_usd'] = market_data[2]['total_collateral_usd']
        contract['prev_total_debt'] = market_data[2]['total_debt']
        contract['prev_n_loans'] = market_data[2]['n_loans']
        contract['snapshot_data'] = market_data
        link = CRVUSD_MARKET_LINKS[address]['link']
        name = CRVUSD_MARKET_LINKS[address]['name']
        contract['marketName'] = name
        image_link, coin_image_html_dict = get_coin_image_url(contract['collateral_token']['address'], 'ethereum', coin_image_html_dict)
        contract['nameLink'] = make_name_link_html(name, link, image_link)
        crvusd_markets[address] = contract

    url = "/v1/dao/fees/crvusd/weekly"
    data = prices_api_request(url)
    for fee in data['fees']:
        address = fee['controller']
        ts = epoch_time_formatter(fee['timestamp'])
        if epoch_end - ts < 7*86400:
            continue
        if 'fees' not in crvusd_markets[address]:
            crvusd_markets[address]['fees'] = fee['fees_usd']
        elif 'prev_fees' not in crvusd_markets[address]:
            crvusd_markets[address]['prev_fees'] = fee['fees_usd']
    
    # Old sfrxETH pool doesn't have fees in prices API, need to estimate them from the rate and debt
    address = '0x8472A9A7632b173c8Cf3a86D3afec50c35548e76'
    crvusd_markets[address]['fees'] = (crvusd_markets[address]['total_debt'] * crvusd_markets[address]['rate']) /365 * 7
    crvusd_markets[address]['prev_fees'] = (crvusd_markets[address]['prev_total_debt'] * crvusd_markets[address]['prev_rate']) /365 * 7

    # Aggregate all the markets into a single 'all' market
    crvusd_markets['all'] = { \
        'fees': 0, \
        'total_debt': 0, \
        'rate': 0, \
        'collateral_amount_usd': 0, \
        'prev_fees': 0, \
        'prev_total_debt': 0, \
        'prev_rate': 0, \
        'prev_collateral_amount_usd': 0, \
        'n_loans': 0, \
        'prev_n_loans': 0, \
    }
    
    for address in crvusd_markets:
        market = crvusd_markets[address]
        if address == 'all':
            continue
        crvusd_markets['all']['fees'] += market['fees']
        crvusd_markets['all']['total_debt'] += market['total_debt']
        crvusd_markets['all']['collateral_amount_usd'] += market['collateral_amount_usd']
        crvusd_markets['all']['prev_fees'] += market['prev_fees']
        crvusd_markets['all']['prev_total_debt'] += market['prev_total_debt']
        crvusd_markets['all']['prev_collateral_amount_usd'] += market['prev_collateral_amount_usd']
        crvusd_markets['all']['prev_rate'] += market['prev_rate'] * market['prev_total_debt']
        crvusd_markets['all']['rate'] += market['rate'] * market['total_debt']
        crvusd_markets['all']['n_loans'] += market['n_loans']
        crvusd_markets['all']['prev_n_loans'] += market['prev_n_loans']

    link = CRVUSD_MARKET_LINKS['all']['link']
    name = CRVUSD_MARKET_LINKS['all']['name']
    crvusd_markets['all']['marketName'] = name
    crvusd_markets['all']['nameLink'] = make_name_link_html(name, link)
    crvusd_markets['all']['rate'] /= crvusd_markets['all']['total_debt']
    crvusd_markets['all']['prev_rate'] /= crvusd_markets['all']['prev_total_debt']

    return crvusd_markets

prices_crvusd_markets = prices_get_crvusd_markets(coin_image_html_dict)

In [42]:
def prices_get_pegkeeper_debt():
    url = "/v1/crvusd/pegkeepers/ethereum"
    data = prices_api_request(url)
    debt = 0
    for pk in data['keepers']:
        debt += pk['total_debt']
    return debt

pk_debt = prices_get_pegkeeper_debt()

In [43]:
def prices_get_scrvusd_snapshots(crv_markets_all, start, end):
    url = f"/v1/crvusd/savings/yield?agg_number=1&agg_units=hour&start={start}&end={end}"
    data = prices_api_request(url)
    scrvusd_data = {
        'prev_staked_crvusd': data['data'][0]['assets'],
        'staked_crvusd': data['data'][-1]['assets'],
        'prev_apy': data['data'][0]['proj_apy'],
        'cur_apy': data['data'][-1]['proj_apy'],
        'prev_staking_ratio': data['data'][0]['assets'] / crv_markets_all['prev_total_debt'],
        'staking_ratio': data['data'][-1]['assets'] / crv_markets_all['total_debt'],
    }
    return scrvusd_data

prices_scrvusd_data = prices_get_scrvusd_snapshots(prices_crvusd_markets['all'], epoch_start, int(time.time()))

In [20]:
def prices_get_crvusd_peg(start, end):
    url = f"/v1/usd_price/ethereum/0xf939e0a03fb07f59a73314e73794be0e57ac1b4e/history?start={start}&end={end}"
    data = prices_api_request(url)
        
    return {'price': data['data'][-1]['price'], 'prev_price': data['data'][0]['price']}

prices_crvusd_peg = prices_get_crvusd_peg(epoch_start, int(time.time()))

In [21]:
scrvusd_image_link, coin_image_html_dict = get_coin_image_url(CRVUSD_ADDR_LIST['scrvUSD'], 'ethereum', coin_image_html_dict)
crvusd_image_link, coin_image_html_dict = get_coin_image_url(CRVUSD_ADDR_LIST['crvUSD'], 'ethereum', coin_image_html_dict)

crvusd_metrics = []

# crvusd supply
metric_name = make_metric_name_with_images([
    {'type': 'img_html', 'content': crvusd_image_link},
    {'type': 'text', 'content': 'crvUSD Supply'}
])
crvusd_metrics.append(make_metric_html(metric_name, prices_crvusd_markets['all']['total_debt'], prices_crvusd_markets['all']['prev_total_debt']))

# scrvusd apy
metric_name = make_metric_name_with_images([
    {'type': 'img_html', 'content': scrvusd_image_link},
    {'type': 'text', 'content': 'scrvUSD Yield'}
])
value = f"{format_number(prices_scrvusd_data['cur_apy'], 4, 2)}% APY"
crvusd_metrics.append(make_metric_html(
    metric_name, 
    prices_scrvusd_data['cur_apy'], 
    prices_scrvusd_data['prev_apy'],
    showPercentChange=False,
    valuePostfix="%",
    customValueFormat=value
))

# staked crvusd ratio
metric_name = make_metric_name_with_images([
    {'type': 'img_html', 'content': crvusd_image_link},
    {'type': 'text', 'content': 'crvUSD in'},
    {'type': 'img_html', 'content': scrvusd_image_link},
    {'type': 'text', 'content': 'scrvUSD'}
])
value = f"{format_number(prices_scrvusd_data['staking_ratio']*100, 4, 2)}%"
crvusd_metrics.append(make_metric_html(
    metric_name,
    prices_scrvusd_data['staking_ratio']*100,
    prices_scrvusd_data['prev_staking_ratio']*100,
    showPercentChange=False,
    valuePostfix="%",
    changeFormatting=",.2f",
    customValueFormat=value
))


# crvusd peg
metric_name = make_metric_name_with_images([
    {'type': 'img_html', 'content': crvusd_image_link},
    {'type': 'text', 'content': 'crvUSD Peg'}
])
value = f"${prices_crvusd_peg['price']:.4f}"
crvusd_metrics.append(make_metric_html(
    metric_name,
    prices_crvusd_peg['price'],
    prices_crvusd_peg['prev_price'],
    showPercentChange=False,
    valuePrefix="$",
    changeFormatting=",.4f",
    customValueFormat=value
))

crvUSD_metrics_html = df_to_html(pd.DataFrame(crvusd_metrics), classes='curve-table')

display_curve_table(crvUSD_metrics_html, HTML_STYLING)

Metric,Value,Change
crvUSD Supply,$77.9M,-0.73%
scrvUSD Yield,15.87% APY,+1.43%
crvUSD inscrvUSD,24.48%,+0.97%
crvUSD Peg,$0.9980,-$0.0001


In [44]:
## crvUSD Loan Stats
crvusd_loan_metrics = []

# crvusd rate
metric_name = f"""Avg. Borrow Rate"""
value = f"{format_number(prices_crvusd_markets['all']['rate']*100, 4, 3)}%"
crvusd_loan_metrics.append(make_metric_html(
    metric_name,
    prices_crvusd_markets['all']['rate']*100,
    prices_crvusd_markets['all']['prev_rate']*100,
    showPercentChange=False,
    valuePostfix="%",
    customValueFormat=value,
))

# borrowed
crvusd_loan_metrics.append(make_metric_html('crvUSD Borrowed', prices_crvusd_markets['all']['total_debt'], prices_crvusd_markets['all']['prev_total_debt']))

# crvusd collateral
crvusd_loan_metrics.append(make_metric_html(
    'Collateral',
    prices_crvusd_markets['all']['collateral_amount_usd'],
    prices_crvusd_markets['all']['prev_collateral_amount_usd']
))

# total loans
crvusd_loan_metrics.append(make_metric_html(
    'Loans', 
    prices_crvusd_markets['all']['n_loans'], 
    prices_crvusd_markets['all']['prev_n_loans'],
    showPercentChange=False,
    changeFormatting=",.0f",
    customValueFormat=prices_crvusd_markets['all']['n_loans']
))

# total fees
crvusd_loan_metrics.append(make_metric_html(
    'Fees',
    prices_crvusd_markets['all']['fees'],
    prices_crvusd_markets['all']['prev_fees']
))

# Pegkeeper debt
metric_name = f"""Pegkeeper Debt"""
value = f"${format_number(pk_debt, 4, 3)}"
change = '-'
crvusd_loan_metrics.append({'Metric': metric_name, 'Value': value, 'Change': change})

crvusd_loan_metrics_html = df_to_html(pd.DataFrame(crvusd_loan_metrics), classes='curve-table')

display_curve_table(crvusd_loan_metrics_html, HTML_STYLING)

Metric,Value,Change
Avg. Borrow Rate,14.66%,-5.02%
crvUSD Borrowed,$77.4M,-1.35%
Collateral,$140.9M,-11.39%
Loans,642,+7
Fees,$188.7k,-32.04%
Pegkeeper Debt,$0,-


In [45]:
full_report_md = f"""## Market Overview

This weekly update highlights the most compelling opportunities across the Curve ecosystem.

> Risk Advisory: References to specific pools or Llamalend markets do not constitute endorsements of their safety. We strongly encourage conducting personal risk analysis before engaging with any pools or markets. Please review our detailed risk disclaimers: [Pool Risk Disclaimer](https://resources.curve.fi/risks-security/risks/pool/), [Llamalend Risk Disclaimer](https://resources.curve.fi/risks-security/risks/lending/), [crvUSD Risk Disclaimer](https://resources.curve.fi/risks-security/risks/crvusd/).

## Top Yields

*Note: for pools with less than $100k TVL, the yield has been calculated as if the pool has $100k TVL.*

### Top Performing Pools by Asset Class

{top_pool_yields_html}

### crvUSD & scrvUSD Yield Opportunities

{top_crvusd_yields_html}

## Ecosystem Metrics

*All changes (shown with + or -) represent the difference compared to values from one week ago.*

### crvUSD & scrvUSD Performance

{crvUSD_metrics_html}

### Pool Statistics

{pool_metrics_html}

### crvUSD Loan Metrics

{crvusd_loan_metrics_html}

### LlamaLend Performance

{lending_metrics_html}

## Notable Pool Activity

### Highest Volume Pools

{pool_highest_volume_html}

### Recently Launched Pools

{pool_biggest_new_pools_html}

### Highest Fee Generating Pools

{pool_biggest_fees_html}

### Most Profitable Pools

*Note: Minimum $100k TVL, profitability is measured by fees per $1 of liquidity*

{pool_most_profitable_html}

---

***Curve's ecosystem continues to expand rapidly, welcoming new teams and pools weekly. If you're interested in launching a pool, lending market, or simply want to connect with the community, join us on [Telegram](https://t.me/curvefi) or [Discord](https://discord.gg/twUngQYz85).**
"""

with open('weekly_report.md', 'w') as f:
    f.write(full_report_md)
    f.close()

with open('weekly_report_html_styling.css', 'w') as f:
    f.write(HTML_STYLING)
    f.close()